In [1]:
import pandas as pd
import tabula as tb
import glob
import re
import shutil, os
import string
import random
import subprocess

# Preprocessing functions for any kind of invoice.

In [2]:
# invoice Classification 
path = "Factures_all/*"
for path in glob.glob(path):
    try:
        list_tab = tb.read_pdf(path, pages = 'all')
        name = re.findall('[0-9]{5}.+', path)
        if list_tab[0].shape[1] == 5 or list_tab[0].shape[1] == 6:
            shutil.move(path, 'Factures/Type1/' + name[0])
        if list_tab[0].shape[1] == 2 or list_tab[0].shape[1] == 3:
            shutil.move(path, 'Factures/Type2/' + name[0])
        if list_tab[0].shape[1] == 8 or list_tab[0].shape[1] == 8:
            shutil.move(path, 'Factures/Type3/' + name[0])
    except(IndexError,subprocess.CalledProcessError):
        print(path)
    
# shape type1 = 5/6
# shape type2 = 2/3
# shape type3 = 8

Factures_all\69004 Lyon- La petite Cocotte- janvier 2020- Facture 2662.pdf
Factures_all\69440 Mornant- Cathy et Bruno- Octobre 2020- Facture 2886.pdf
Factures_all\78800 Houilles- Fidéci- Mai 2020- Facture 2762.pdf


Error from tabula-java:
Error: Error: Header doesn't contain versioninfo




Factures_all\95110 Sannois- JLC Concept- mai 2020- Facture 2761.pdf
Factures_all\facture avec mention manuscrite GIJOM JUIN 2020 001.jpg


In [3]:
def client_gen():
    size = 3
    num = string.digits
    client_num = "CL42"+"".join(random.sample(num,size))
    return client_num

In [4]:
# functions for Type1 invoice
def preprocess_pdf(path):
    list_tab = tb.read_pdf(path, pages = 'all')

    if len(list_tab[0]) == 1:
        ref = list_tab[0]
        client_num = ref.iloc[0,2]
        fact_num = ref.iloc[0,0]
        date = ref.iloc[0,1]
        df = list_tab[1]

    else :
        df = list_tab[0]
        i = 0
        while df.iloc[i,0] != 'Numéro Date':
            i += 1
        i +=1
        # create features 
        client_num = df.iloc[i,1]
        fact_date = df.iloc[i,0]
        fact_num = fact_date.split()[0]
        date = fact_date.split()[1]

    cp = re.findall('[0-9]{5}', path)
# Cleaning the dataframe and transform commas in dots for convertion
    mask = df[df.columns[0]].str[0:2] == str('AR')
    df1 = df[mask].dropna(thresh=.7,axis=1)
    for i in range(len(df1.columns)):
            df1.iloc[:,i] = df1.iloc[:,i].apply(lambda x: x.replace(",","."))
            df1.iloc[:,i] = df1.iloc[:,i].apply(lambda x: x.replace("1S"," "))
    if len(df1.columns) != 2:
        df1[df1.columns[2]] = df1[df1.columns[2]].apply(lambda x: x.replace("1 ","1"))
        df1[df1.columns[2]] = df1[df1.columns[2]].apply(lambda x: x.replace("2 ","2"))
# spliting columns
    if len(df1.columns) == 3:
        df1[['Code','Desc']] = df1.iloc[:,0].str.split(n=1, expand = True)
        df1[['Qté', "PUHT"]] = df1.iloc[:,1].str.split(expand=True)
        df1[['Montant', "TVA"]] = df1.iloc[:,2].str.split(expand = True)

    elif len(df1.columns) == 6:
        df1.rename(columns = {df1.columns[0]:'Code',df1.columns[1]: 'Desc',df1.columns[2]:'Qté',df1.columns[3]:'PUHT',df1.columns[4]:'Montant',df1.columns[5]:'TVA'},inplace=True)
        
    else:
        df1[['Code','Desc']] = df1.iloc[:,0].str.split(n = 1, expand = True)
        df1[['Qté','PUHT','Montant','TVA']] = df1.iloc[:,1].str.split(n = 3, expand = True)
        

# adding new features in dataframe
    df1['client_num'] = client_num
    df1['fact_num'] = fact_num
    df1['date'] = date
    df1['CP'] = cp[0]

# drop unuse columns
    if len(df1.columns) > 8 and len(df1.columns) != 12 and len(df1.columns) != 10:
        df1 = df1.drop(list(df1.columns[[0,1,2]]), axis=1)
    elif len(df1.columns) == 12:
        df1 = df1.drop(list(df1.columns[[0,1]]), axis=1)
        
# Converting type of columns
    df1[["Qté","PUHT","Montant","TVA"]] = df1[["Qté","PUHT","Montant","TVA"]].astype(float)
    df1['date'] = df1['date'].astype('datetime64[ns]')
    df1['CP'] = df1['CP'].astype(str)
    return df1
    
# functions for Type2 invoice
def preprocess_pdf2(path):
    list_tab = tb.read_pdf(path, pages = 'all')
    # test different kind of shape to find the correct extractor
    if list_tab[0].shape[0] != 0:
        df = list_tab[0]
        df = df.iloc[:,[0,1]]
        mask = df.iloc[:,1].str.contains('[FA]{2}[0-9]{7}', regex = True, na=False)
        fact_num = df[mask].iloc[:,1].values[0]
        mask_2 = df.iloc[:,0].str.contains('[CL]{1}[0-9]{5}', regex = True, na=False)
        client_num = df[mask_2].iloc[:,0].values[0].split()[0]
        date = df[mask_2].iloc[:,0].values[0].split()[1]


    elif list_tab[0].shape[0] == 0 and list_tab[1].shape[1] <2:
        list_tab[1] =  list_tab[2]
        client_num = client_gen()
        date = '01/01/2000'
        fact_num = 'FA' + '0000' + path.split(" ")[-1].split(".")[0]

    else:
        client_num = client_gen()
        date = '01/01/2000'
        fact_num = 'FA' + '0000' + path.split(" ")[-1].split(".")[0]
    cp = re.findall('[0-9]{5}', path)

    
    df2 = list_tab[1]
    mask = df2[df2.columns[0]].str[0:2] == str('AR')
    df2 = df2[mask].dropna(thresh=.7,axis=1)
    df2 = df2.reset_index(drop=True)
    mask = df2[df2.columns[0]].str.contains('[AR].......', regex = True, na=False)
    if mask.sum() >= 1:
        temp = df2[mask].iloc[0,:].str.split('\r',expand=True).T
        df2 = df2.drop(df2[mask].index[0],0)
        fusion = [df2,temp]
        df3 = pd.concat(fusion)
        df3 = df3.reset_index(drop=True)
    else:
        df3 = df2.copy()

    mask = df3[df3.columns[2]].str.contains('^\d......', regex = True, na=False)
    if mask.sum() >= 1:
        temp = df3[mask].iloc[0,:].str.split('\r',expand=True).T
        df3 = df3.drop(df3[mask].index[0],0)
        fusion = [df3,temp]
        df4 = pd.concat(fusion)
        df4 = df4.reset_index(drop=True)
    else:
        df4 = df3.copy()
    if len(df4.columns) > 6:
        df4 = df4.drop([df4.columns[4]], axis=1)
    df4 = df4.dropna()
    for i in range(len(df4.columns)):
        df4.iloc[:,i] = df4.iloc[:,i].apply(lambda x: x.replace(",","."))
    df4['client_num'] = client_num
    df4['fact_num'] = fact_num
    df4['date'] = date
    df4['CP'] = cp[0]
    df4.rename(columns = {df4.columns[0]:'Code', df4.columns[1]: 'Desc', df4.columns[2]:'Qté', df4.columns[3]:'PUHT', df4.columns[4]:'Montant',df4.columns[5]:'TVA'},inplace=True)
    df4["Montant"] = df4["Montant"].apply(lambda x: x.replace(" ",""))
    df4[["Qté","PUHT","Montant","TVA"]] = df4[["Qté","PUHT","Montant","TVA"]].astype(float)
    df4['date'] = df4['date'].astype('datetime64[ns]')
    df4['CP'] = df4['CP'].astype(str)
    return df4

# functions for Type3 invoice
def preprocess_pdf3(path):
    list_tab = tb.read_pdf(path, pages = 'all')
# select the first table and create features
    if list_tab[1].shape[0] == 0:
        df = list_tab[2]
    else:
        df = list_tab[1]

    mask = df.iloc[:,1].str.contains('[FA]{2}[0-9]{7}', regex = True, na=False)
    fact_num = df[mask].iloc[:,1].values[0]
    mask = df.iloc[:,1].str.contains('[0-9]{2}.[0-9]{2}.[0-9]{4}', regex = True, na=False)
    date = df[mask].iloc[:,1].values[0]
    client_num = client_gen() 
    cp = re.findall('[0-9]{5}', path)
# select the second table clean and add features
    df2= list_tab[0]
    mask = df2[df2.columns[0]].str[0:2] == str('AR')
    df2 = df2[mask].dropna(thresh=.7,axis=1)
    df2 = df2.dropna()
    for i in range(len(df2.columns)):
               df2.iloc[:,i] = df2.iloc[:,i].apply(lambda x: x.replace(",","."))
    if list_tab[1].shape[0] == 0 and df2.columns[1] == 'Désignation Qté':
        df2[['Desc', "Qté"]] = df2.iloc[:,1].str.rsplit(" ",n=1,expand=True)
        df2 = df2.drop([df2.columns[1]],axis=1)
        df2.rename(columns = {df2.columns[0]:'Code',df2.columns[1]: 'PUHT',df2.columns[2]:'Montant',df2.columns[3]:'TVA',df2.columns[4]:'Desc',df2.columns[5]:'Qté'},inplace=True)
    else:
        df2.rename(columns = {df2.columns[0]:'Code',df2.columns[1]: 'Desc',df2.columns[2]:'Qté',df2.columns[3]:'PUHT',df2.columns[4]:'Montant',df2.columns[5]:'TVA'},inplace=True)

    if (df2[df2.columns[1]].str[-9:-1] == 'découvri').sum() == 1:
        mask = df2[df2.columns[1]].str[-9:-1] == 'découvri'
        df2[mask].index[0]
        df2.drop(df2[mask].index[0], inplace=True)
        
    df2['client_num'] = client_num
    df2['fact_num'] = fact_num
    df2['date'] = date
    df2[["Qté","PUHT","Montant","TVA"]] = df2[["Qté","PUHT","Montant","TVA"]].astype(float)
    df2['date'] = df2['date'].astype('datetime64[ns]')
    df2['CP'] = cp[0]
    df2['CP'] = df2['CP'].astype(str)
    
    return df2

# LOAD INVOICES

In [5]:
path = "Factures/Type1/*"
list_path = []
for path in glob.glob(path):
    list_path.append(path)

path2 = "Factures/Type2/*"
list_path_2 = []
for path in glob.glob(path2):
    list_path_2.append(path)

path3 = "Factures/Type3/*"
list_path_3 = []
for path in glob.glob(path3):
    list_path_3.append(path)
    
print('Facture de Type 1:',len(list_path))
print('Facture de Type 2:',len(list_path_2))
print('Facture de Type 3:',len(list_path_3))

Facture de Type 1: 134
Facture de Type 2: 123
Facture de Type 3: 6


# APPLY CONVERSION

In [6]:
# Type 1 invoice
path_error_type1= [] # use to catch Error
facture = [] # the list with all invoice

for path in list_path:
    try:
        facture.append(preprocess_pdf(path))
        print('conversion OK:',path)
    except (IndexError):
        print('conversion ERROR',path)
        path_error_type1.append(path)
    except (KeyError):
        print('conversion ERROR',path)
        path_error_type1.append(path)
print("Nombre d'erreurs sur le type 1:",len(path_error_type1))
print("Nombre total de factures converties :",len(facture))

conversion OK: Factures/Type1\02600 Villers  Cotterets- Ma Boutique Tany- juin 2020- Facture 2786.pdf
conversion OK: Factures/Type1\04300 Forcalquier- Petra Patrimonia- Marjorie Thiers- mai 2020- Facture 2727.pdf
conversion OK: Factures/Type1\05000 Gap- AS- fevrier 2020- Facture 2679.pdf
conversion OK: Factures/Type1\06370 Mouans Sartoux- Boomerang- janvier 2020- Facture 2661.pdf
conversion OK: Factures/Type1\06600 Antibes- Herbularium la Melisse- octobre 2019- Facture 2567.pdf
conversion OK: Factures/Type1\08000 Charleville Mezieres- La Malinerie- novembre 2020- Facture 2921.pdf
conversion OK: Factures/Type1\11000 Carcassonne- AS- fevrier 2020- Facture 2697.pdf
conversion OK: Factures/Type1\11000 Carcassonne- La Ferme- fevrier 2020- Facture 2698.pdf
conversion OK: Factures/Type1\11000 Carcassonne- La Ferme- octobre 2020- Facture 2892.pdf
conversion OK: Factures/Type1\12200 Villefranche de Rouergue- AS- juin 2020- Facture 2770.pdf
conversion OK: Factures/Type1\12200 Villefranche sur Ro

In [7]:
# Type 2 invoice
path_error_type2 = []
facture_2 = []
for path in list_path_2:
    try:
        facture_2.append(preprocess_pdf2(path))
        print('conversion OK:',path)
    except IndexError as index:
        print('conversion ERROR',index," ",path )
        path_error_type2.append(path)
print("Nombre d'erreurs sur le type 2:",len(path_error_type2))
print("Nombre total de factures  converties :",len(facture_2))

conversion OK: Factures/Type2\05290 Vallouise- Les Vaudois- juin 2020- Facture 2789.pdf
conversion OK: Factures/Type2\08000 Charleville Mezieres- La Malinerie- mars 2020- Facture 2718.pdf
conversion OK: Factures/Type2\11100 Narbonne- Sans BPA- janvier 2020- Facture 2665.pdf
conversion OK: Factures/Type2\11100 Narbonne- Sans BPA-septembre 2020- Facture 2879.pdf
conversion OK: Factures/Type2\17000 La Rochelle- Aux Couleurs Nouvelles- fevrier 2020 - Facture 2683.pdf
conversion OK: Factures/Type2\17000 La Rochelle- Marmites et Cocottes- fevrier 2020- Facture 2706.pdf
conversion OK: Factures/Type2\17000 Royan- Juglas- juillet 2020- Facture 2820.pdf
conversion OK: Factures/Type2\18700 Aubigny sur Nere- ET Caetera- fevrier 2020- Facture 2681.pdf
conversion OK: Factures/Type2\21200 Beaune- Athenaeum- fevrier 2020- Facture 2680.pdf
conversion OK: Factures/Type2\22300 Lannion- Capucine- juin 2020- Facture 2791.pdf
conversion OK: Factures/Type2\22300 Lannion- Mele Capucine- novembre 2020- Facture

In [8]:
# Type 3 invoice
path_error_type3 = []
facture_3 = []
for path in list_path_3:
    try:
        facture_3.append(preprocess_pdf3(path))
        print('conversion OK:',path)
    except (ValueError,IndexError):
        print('conversion ERROR',path)
        path_error_type3.append(path)
print("Nombre d'erreurs sur le type 3:",len(path_error_type3))
#print("Nombre total d'erreurs de conversion:",len(path_error_type3)+len(path_error_type2)+len(path_error_type1))
#print("Nombre total de factures converties :",len(facture_3 + facture_2 + facture))

conversion OK: Factures/Type3\14000 Caen- La Maison du Vrac- mai 2020- Facture 2750.pdf
conversion OK: Factures/Type3\46110 Vayrac- Besoin d'Envies- mai 2020- Facture 2763.pdf
conversion OK: Factures/Type3\55200 Commercy- Majon Cabinet de Curiosité- mai 2020- Facture 2734.pdf
conversion OK: Factures/Type3\96000 Belgique -Greenweez BELGIQUE- juin 2020- Masques 2- Facture 2776.pdf
conversion OK: Factures/Type3\97000 Cesena- Italie -Roberta Paganelli-  février 2020- Facture 2668.pdf
conversion OK: Factures/Type3\97000- Cesena- Paganelli- juin 2020- Facture 2777.pdf
Nombre d'erreurs sur le type 3: 0


# CREATING DATAFRAME AND SAVE

In [9]:
df = pd.concat(facture)

In [10]:
df.reset_index(drop=True, inplace=True)

In [11]:
df = pd.concat(facture)
df.head()

,Code,Desc,Qté,PUHT,Montant,TVA,client_num,fact_num,date,CP
5,AR00100,SAS CITRON BLEU GM,5.0,12.5,62.5,20.0,CL00222,FA00002786,2020-06-16,02600
6,AR00098,SAS CLUB VICHY Rouge GM,5.0,12.5,62.5,20.0,CL00222,FA00002786,2020-06-16,02600
7,AR00084,SAS FEUILLAGE Vert GM,5.0,12.5,62.5,20.0,CL00222,FA00002786,2020-06-16,02600
8,AR00097,SAS MELI-MELO GM,10.0,12.5,125.0,20.0,CL00222,FA00002786,2020-06-16,02600
9,AR00012,SAS PETITS PIMENTS GM,5.0,12.5,62.5,20.0,CL00222,FA00002786,2020-06-16,02600


In [12]:
df2 = pd.concat(facture_2)
df2.head()

,Code,Desc,Qté,PUHT,Montant,TVA,client_num,fact_num,date,CP
0,AR00098,SAS CLUB VICHY Rouge GM,6.0,12.5,75.0,20.0,CL00512,FA00002789,2020-06-19,05290
1,AR00084,SAS FEUILLAGE Vert GM,6.0,12.5,75.0,20.0,CL00512,FA00002789,2020-06-19,05290
2,AR00009,SAS HERBES FOLLES GM,6.0,12.5,75.0,20.0,CL00512,FA00002789,2020-06-19,05290
3,AR00097,SAS MELI-MELO GM,6.0,12.5,75.0,20.0,CL00512,FA00002789,2020-06-19,05290
4,AR00012,SAS PETITS PIMENTS GM,6.0,12.5,75.0,20.0,CL00512,FA00002789,2020-06-19,05290


In [13]:
df3 = pd.concat(facture_3)
df3.head()

,Code,Desc,Qté,PUHT,Montant,TVA,client_num,fact_num,date,CP
4,AR00084,SAS FEUILLAGE Vert GM,5.0,12.5,62.5,20.0,CL42871,FA00002750,2020-05-18,14000
5,AR00092,SAS NID D'ABEILLE GM,5.0,12.5,62.5,20.0,CL42871,FA00002750,2020-05-18,14000
6,AR00012,SAS PETITS PIMENTS GM,5.0,12.5,62.5,20.0,CL42871,FA00002750,2020-05-18,14000
7,AR00101,SAS ZEN GM,5.0,12.5,62.5,20.0,CL42871,FA00002750,2020-05-18,14000
8,AR00012,SAS PETITS PIMENTS GM modèle Expo + PLV,1.0,0.0,0.0,20.0,CL42871,FA00002750,2020-05-18,14000


In [14]:
data = pd.concat([df,df2,df3])

In [15]:
data.head()

,Code,Desc,Qté,PUHT,Montant,TVA,client_num,fact_num,date,CP
5,AR00100,SAS CITRON BLEU GM,5.0,12.5,62.5,20.0,CL00222,FA00002786,2020-06-16,02600
6,AR00098,SAS CLUB VICHY Rouge GM,5.0,12.5,62.5,20.0,CL00222,FA00002786,2020-06-16,02600
7,AR00084,SAS FEUILLAGE Vert GM,5.0,12.5,62.5,20.0,CL00222,FA00002786,2020-06-16,02600
8,AR00097,SAS MELI-MELO GM,10.0,12.5,125.0,20.0,CL00222,FA00002786,2020-06-16,02600
9,AR00012,SAS PETITS PIMENTS GM,5.0,12.5,62.5,20.0,CL00222,FA00002786,2020-06-16,02600


In [16]:
data.to_csv('factures_csv.csv', index=None)